In [ ]:
# !pip install langchain
# !pip install -U langchain-openai
# !pip install -U langchain_neo4j
# !pip install langchain_community
# !pip install neo4j
# !pip install shapely
# !pip install -U pyproj
# !pip install tqdm
# !pip install langchain-community
# !pip install -U pandas

# !pip install -U langchain-neo4j

In [15]:
# import json
# from tqdm import tqdm
from dotenv import load_dotenv
load_dotenv()

import os

from langchain_neo4j import Neo4jGraph
from langchain.chat_models import init_chat_model
from langgraph.graph import START, StateGraph
from langchain_core.prompts import PromptTemplate
from typing_extensions import List, TypedDict


import psycopg2
from neo4j import GraphDatabase
from decimal import Decimal
from shapely import wkb
from pyproj import CRS, Transformer


In [16]:

load_dotenv()

keys = ["NEO4J_USER", "NEO4J_PASSWORD","NEO4J_URI_LOCAL", "NEO4J_USER_LOCAL", "NEO4J_PASSWORD_LOCAL"]
for key in keys:
    if key not in os.environ:
        raise Exception(f"Key '{key}' not found not in .env")
    
print("Credentials from .env file successfully loaded")

NEO4J_LOCAL_DATABASE = "busoppdse203"

Credentials from .env file successfully loaded


In [17]:
graph = Neo4jGraph(url="bolt://localhost:7687",    username='neo4j',password='admin123',database=NEO4J_LOCAL_DATABASE )
graph.query("MATCH (n) RETURN n LIMIT 1;")


[{'n': {'code': 'CA', 'name': 'Calfironia', 'id': 1}}]

In [19]:
# Initialize the LLM
model_ver = "gpt-4o"
model_provider = 'openai'
model = init_chat_model(model_ver, model_provider=model_provider)


In [20]:
# Define state for application
class State(TypedDict):
    question: str
    context: List[dict]
    answer: str


In [49]:
# Retrieve context 
# def retrieve(state: State):
#     context = graph.query("CALL db.schema.visualization()")
#     return {"context": context}




def retrieve(state: State):
    schema_map = graph.query("CALL apoc.meta.schema()")
    
    context = schema_map 
    return {"context": context}

In [50]:
# Create a prompt
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}

Answer:"""

prompt = PromptTemplate.from_template(template)



In [47]:
# Generate the answer based on the question and context
def generate(state: State):
    messages = prompt.invoke({"question": state["question"], "context": state["context"]})
    response = model.invoke(messages)
    return {"answer": response.content}







In [52]:
# Define application steps
workflow = StateGraph(State).add_sequence([retrieve, generate])
workflow.add_edge(START, "retrieve")
app = workflow.compile()


In [53]:
entity_type = 'BlockGroup'
# question = "How is the graph structured?"



# question = "What questions can you answer"
question = f"What attributes are available on a {entity_type} node"
# question = "Are any cities contained in cities"

# question = 'What is the context'

response = app.invoke({"question": question})
print("Answer:", response["answer"])

Answer: The attributes available on a BlockGroup node are:

- avghinc_cy (type: INTEGER)
- n01_bus (type: FLOAT)
- crmcytotc (type: INTEGER)
- n37_sales (type: FLOAT)
- male25 (type: INTEGER)
- s22_bus (type: FLOAT)
- aggregatio (type: STRING)
- gini_fy (type: FLOAT)
- gtype (type: INTEGER)
- x1001fy_x (type: FLOAT)
- statefp (type: INTEGER)
- bbox (type: LIST)
- fem25 (type: INTEGER)
- population (type: FLOAT)
- ctblockgroup (type: INTEGER)
- di100_cy (type: INTEGER)
- fem30 (type: INTEGER)
- source_cou (type: STRING)
- totpop_cy (type: INTEGER)
- medhinc_cy (type: INTEGER)
- indmanu_cy (type: INTEGER)
- s16_sales (type: FLOAT)
- s23_emp (type: FLOAT)
- di150_cy (type: INTEGER)
- countyfp (type: INTEGER)
- tractce (type: INTEGER)
- male30 (type: INTEGER)
- blkgrpce (type: INTEGER)
- apportionm (type: FLOAT)
- male35 (type: INTEGER)
- ogc_fid (type: INTEGER)
- geojson_geom (type: STRING)
- fem35 (type: INTEGER)
- n35_bus (type: FLOAT)
- n14_bus (type: FLOAT)
- x1024_x (type: FLOAT)
